### P0 Code Generator for WASM

In [3]:
import nbimporter; nbimporter.options["only_defs"] = False
from SC import TIMES, DIV, MOD, PLUS, MINUS, AND, OR, EQ, NE, LT, GT, LE, GE, \
     NOT, CARD, COMPLEMENT, UNION, INTERSECTION, ELEMENT, SUBSET, SUPERSET, SET, \
     mark, ARROW
from ST import indent, Var, Const, Type, Proc, StdProc, Int, Bool, Array, PartialArray, \
     Record, Set
from ST import find

def genProgStart():
    global curlev, memsize, asm, asmPartArr
    curlev, memsize = 0, 0
    asm = ['(module',
           '(import "P0lib" "write" (func $write (param i32)))',
           '(import "P0lib" "writeln" (func $writeln))',
           '(import "P0lib" "read" (func $read (result i32)))']
    asmPartArr = []

def genBool(b: Bool):
    b.size = 1; return b

def genInt(i: Int):
    i.size = 4; return i

def genRec(r: Record):
    s = 0
    for f in r.fields:
        f.offset, s = s, s + f.tp.size
    r.size = s
    return r

def genArray(a: Array):
    a.size = a.length * a.base.size
    return a

def genPartialArray(a: PartialArray):
    # null values for non-existing values for some keys
    a.size = a.length * a.base.size
    return a

def genSet(s: Set):
    if s.lower < 0 or s.lower + s.length > 32:
        mark('WASM: set too large')
    s.size = 4; return s

Global = 0; Stack = -1; MemInd = -2; MemAbs = -3

def genGlobalVars(sc, start):
    print("======genGlobalVars sc: ", sc, ", start: ", start)
    for x in range(len(sc)):
        print("===sc[", x, "]", sc[x])
    global memsize
    for i in range(start, len(sc)):
        if type(sc[i]) == Var:
            if sc[i].tp in (Int, Bool) or type(sc[i].tp) == Set:
                asm.append('(global $' + sc[i].name + ' (mut i32) i32.const 0)')
            elif type(sc[i].tp) in (Array, Record):
                sc[i].lev, sc[i].adr, memsize = MemAbs, memsize, memsize + sc[i].tp.size
            elif type(sc[i].tp) == PartialArray:
                sc[i].lev, sc[i].adr = MemAbs, sc[i].adr
                asm.append('(global $' + sc[i].name + ' (mut i32) i32.const ' + str(sc[i].adr) + ')')
            else: mark('WASM: type?')
    
def genLocalVars(sc, start):
    print("======genLocalVars sc: ", sc, ", start: ", start)
    for i in range(start, len(sc)):
        if type(sc[i]) == Var:
            asm.append('(local $' + sc[i].name + ' i32)')
    asm.append('(local $0 i32)') # auxiliary local variable
    return sc[start:]

def loadItem(x):
    print("====loadItem x: ", x)
    if type(x) == Var:
        if x.lev == Global: asm.append('global.get $' + x.name) # global Var
        elif x.lev == curlev: asm.append('local.get $' + x.name) # local Var
        elif x.lev == MemInd: print("==== loadItem MemInd x: ", x); asm.append('i32.load')
        elif x.lev == MemAbs:
            asm.append('i32.const ' + str(x.adr))
            if x.tp in {Int, Bool}: print("==== loadItem MemAbs x: ", x); asm.append('i32.load')
        elif x.lev != Stack: mark('WASM: var level!') # already on stack if lev == Stack
    elif type(x) is tuple:
        varPartialArray, indexValVar = find(x[0]), x[1]
        print("==== loadItem MemInd partial array varPartialArray: ", varPartialArray, ", indexValVar: ", indexValVar)
        if type(varPartialArray.tp) == PartialArray and indexValVar.lev == MemInd: print("==== loadItem MemInd partial array x: ", x); asm.append('i32.load')
    else: asm.append('i32.const ' + str(x.val))

def genVar(x):
    if Global < x.lev < curlev: mark('WASM: level!')
    y = Var(x.tp); y.lev, y.name = x.lev, x.name
    if x.lev == MemAbs: y.adr = x.adr
    return y

def genConst(x):
    # x is Const
    x.lev = None # constants are either not stored or on stack, lev == Stack
    return x

def genUnaryOp(op, x):
    loadItem(x)
    if op == MINUS:
        asm.append('i32.const -1')
        asm.append('i32.mul')
        x = Var(Int); x.lev = Stack
    elif op == CARD:
        asm.append('i32.popcnt')
        x = Var(Int); x.lev = Stack
    elif op == COMPLEMENT:
        u = (1 << x.tp.length) - 1 # x.tp.length 1's
        u = u << x.tp.lower # universe of base type
        asm.append('i32.const ' + hex(u))
        asm.append('i32.xor')
        x = Var(x.tp); x.lev = Stack
    elif op == SET:
        asm.append('local.set $0')
        asm.append('i32.const 1')
        asm.append('local.get $0')
        asm.append('i32.shl')
        x = Var(Set(0, 32)); x.lev = Stack
    elif op == NOT:
        asm.append('i32.eqz')
        x = Var(Bool); x.lev = Stack
    elif op == AND:
        asm.append('if (result i32)')
        x = Var(Bool); x.lev = Stack
    elif op == OR:
        asm.append('if (result i32)')
        asm.append('i32.const 1')
        asm.append('else')
        x = Var(Bool); x.lev = Stack
    elif op == ELEMENT:
        asm.append('local.set $0')
        asm.append('i32.const 1')
        asm.append('local.get $0')
        asm.append('i32.shl')
        x = Var(Int); x.lev = Stack
    elif op in {SUBSET, SUPERSET}:
        asm.append('local.tee $0')
        asm.append('local.get $0')
        x.lev = Stack
    else: mark('WASM: unary operator?')
    return x

def genBinaryOp(op, x, y):
    if op in (PLUS, MINUS, TIMES, DIV, MOD):
        loadItem(x); loadItem(y)
        asm.append('i32.add' if op == PLUS else \
                   'i32.sub' if op == MINUS else \
                   'i32.mul' if op == TIMES else \
                   'i32.div_s' if op == DIV else \
                   'i32.rem_s' if op == MOD else '?')
        x = Var(Int); x.lev = Stack
    elif op in {UNION, INTERSECTION}:
        loadItem(x); loadItem(y)
        asm.append('i32.or' if op == UNION else \
                   'i32.and' if op == INTERSECTION else '?')
        x = Var(x.tp); x.lev = Stack
    elif op == AND:
        loadItem(y) # x is already on the stack
        asm.append('else')
        asm.append('i32.const 0')
        asm.append('end')
        x = Var(Bool); x.lev = Stack
    elif op == OR:
        loadItem(y) # x is already on the stack
        asm.append('end')
        x = Var(Bool); x.lev = Stack
    else: mark('WASM: binary operator?')
    return x

def genRelation(op, x, y):
    loadItem(x); loadItem(y)
    asm.extend(['i32.eq'] if op == EQ else \
               ['i32.ne'] if op == NE else \
               ['i32.lt_s'] if op ==  LT else \
               ['i32.gt_s'] if op == GT else \
               ['i32.le_s'] if op == LE else \
               ['i32.ge_s'] if op == GE else \
               ['i32.and'] if op == ELEMENT else \
               ['i32.and', 'i32.eq'] if op == SUBSET else \
               ['i32.or', 'i32.eq'] if op == SUPERSET else '?')
    x = Var(Bool); x.lev = Stack
    return x

def genIndex(x, y):
    # x[y], assuming x.tp is Array and x is global Var, local Var
    # and y is Const, local Var, global Var, stack Var
    if x.lev == MemAbs and type(y) == Const: 
        x.adr += (y.val - x.tp.lower) * x.tp.base.size
        x.tp = x.tp.base
    else:
        loadItem(y)
        if x.tp.lower != 0:
            asm.append('i32.const ' + str(x.tp.lower))
            asm.append('i32.sub')
        asm.append('i32.const ' + str(x.tp.base.size))
        asm.append('i32.mul')
        if x.lev > 0: asm.append('local.get $' + x.name)
        elif x.lev == MemAbs: asm.append('i32.const ' + str(x.adr))
        asm.append('i32.add')
        x = Var(x.tp.base)
        if x.tp in (Int, Bool) or type(x.tp) == Set: x.lev = MemInd
        else: x.lev = Stack
    return x

def genPartialIndexes(paVar):
    print("=====genPartialIndexes PartialArray: ", paVar.tp)
    global memsize, asmPartArr
    pa = paVar.tp
    for key, val in pa.keyValDict.items():
        print("=====genPartialIndexes key: ", key, ", val: ", val, ", pa.size: ", pa.size)
        asmPartArr.append('i32.const ' + str(memsize + (val[0].val-1)*pa.base.size))
        asmPartArr.append('i32.const ' + str(val[1].val))
        asmPartArr.append('i32.store')
    paVar.adr = memsize
    memsize = memsize + pa.size

def genPartialIndex(x, y):
    # x[y], assuming x.tp is Array and x is global Var, local Var
    # and y is Const, local Var, global Var, stack Var
    print("=====genPartialIndex==== x: ", x, ", y: ", y)
    if x.lev == MemAbs and type(y) == Const: 
        print("=====genPartialIndex==== if 1")
        valVar = Var(x.tp.keyValDict[y.val][1])
        print("====genPartialIndex=== valVar: ", valVar)
        if valVar.tp.tp in (Int, Bool) or type(valVar.tp.tp) == Set: valVar.lev = MemInd
        else: valVar.lev = Stack
        lowerBound = list(x.tp.keyValDict.keys())[0]
        x.adr += (y.val - lowerBound) * x.tp.base.size
        x.tp = x.tp.base
        x = (x, valVar)
        print("=====genPartialIndex==== if end x: ", x)
    else:
        print("=====genPartialIndex==== else 1 x: ", x, ", y: ", y)
        loadItem(y)
        print("=====genPartialIndex==== else 2")
        lowerBound = list(x.tp.keyValDict.keys())[0]
        if lowerBound != 0:
            print("=====genPartialIndex==== else 3")
            asm.append('i32.const ' + str(lowerBound))
            asm.append('i32.sub')
            print("=====genPartialIndex==== else 4")
        print("=====genPartialIndex==== else 5")
        asm.append('i32.const ' + str(x.tp.base.size))
        asm.append('i32.mul')
        print("=====genPartialIndex==== else 6")
        if x.lev > 0: print("=====genPartialIndex==== else 7"); asm.append('local.get $' + x.name); print("=====genPartialIndex==== else 8")
        elif x.lev == MemAbs: print("=====genPartialIndex==== else 9"); asm.append('i32.const ' + str(x.adr)); print("=====genPartialIndex==== else 10")
        print("=====genPartialIndex==== else 11")
        asm.append('i32.add')
        # x = Var(x.tp.base)
        # print("=====genPartialIndex==== else 12")
        # if x.tp in (Int, Bool) or type(x.tp) == Set: x.lev = MemInd
        # else: x.lev = Stack
        print("====genPartialIndex=== after if/else x: ", x)
        valVar = Var(x.tp.keyValDict[y.val][1])
        print("====genPartialIndex=== valVar: ", valVar)
        if valVar.tp.tp in (Int, Bool) or type(valVar.tp.tp) == Set: valVar.lev = MemInd
        else: valVar.lev = Stack
        x = (x.name, valVar)
        print("=====genPartialIndex==== end x: ", x)
    return x

def genSelect(x, f):
    # x.f, assuming x.tp is Record, f is Field, and x.lev is Stack, MemInd or is > 0
    if x.lev == MemAbs: x.adr += f.offset
    elif x.lev == Stack:
        asm.append('i32.const ' + str(f.offset))
        asm.append('i32.add')
    elif x.lev > 0:
        asm.append('local.get $' + x.name) # parameter or local reference
        asm.append('i32.const ' + str(f.offset))
        asm.append('i32.add')
        x.lev = Stack
    else: mark('WASM: select?')
    x.tp = f.tp
    return x

def genLeftAssign(x):
    print("=====genLeftAssign x: ", x)
    if x.lev == MemAbs:
        if type(x.tp) == PartialArray:
            for key, value in x.tp.keyValDict.items():
                if value[1] is None:
                    asm.append('i32.const ' + str(value[0].val))
                    asm.append('i32.const ' + str(1))
                    asm.append('i32.sub')
                    asm.append('i32.const ' + str(x.tp.base.size))
                    asm.append('i32.mul')
                    asm.append('global.get $' + x.name)
                    asm.append('i32.add')
        else:
            asm.append('i32.const ' + str(x.adr))
    elif x.lev > 0 and type(x.tp) in (Array, Record):
        asm.append('local.get $' + x.name)
    elif x.lev > 0 and type(x.tp) == PartialArray:
        for key, value in x.tp.keyValDict.items():
            if value[1] is None:
                asm.append('i32.const ' + str(value[0].val))
                asm.append('i32.const ' + str(1))
                asm.append('i32.sub')
                asm.append('i32.const ' + str(x.tp.base.size))
                asm.append('i32.mul')
                asm.append('local.get $' + x.name)
                asm.append('i32.add')
    return x

def genRightAssign(x):
    print("=====genRightAssign x: ", x, ", x.tp: ", x.tp)
    if type(x.tp) == PartialArray: y = Var(Int()); y.lev = MemInd
    else: loadItem(x); y = Var(x.tp); y.lev = Stack
    return y

def genAssign(x, y):
    print("=====genAssign x: ", x, ", y: ", y)
    if type(x.tp) != PartialArray: loadItem(y)
    if x.lev == Global: asm.append('global.set $' + x.name)
    elif x.lev > 0:
        print("====genAssign if x.lev: ", x.lev)
        if type(x.tp) in (Array, PartialArray, Record):
            asm.append('i32.const ' + str(x.tp.size))
            asm.append('memory.copy')
        else: asm.append('local.set $' + x.name)
    else:
        print("====genAssign else x.lev: ", x.lev)
        if type(x.tp) in (Array, Record):
            asm.append('i32.const ' + str(x.tp.size))
            asm.append('memory.copy')
        elif type(x.tp) == PartialArray:
            asm.append('i32.const ' + str(x.tp.base.size))
            asm.append('memory.copy')
        else: asm.append('i32.store')

def genProgEntry(ident):
    global curlev, asmPartArr
    curlev = curlev + 1
    asm.append('(global $_memsize (mut i32) i32.const ' + str(memsize) + ')')
    asm.append('(func $program')

def genProgExit(x):
    global curlev
    curlev = curlev - 1
    asm.append('(memory ' + str(memsize // 2** 16 + 1) + ')\n(start $program)\n)')
    return '\n'.join(l for l in asm)

def genProcStart(ident, fp, rp):
    global curlev
    if curlev > 0: mark('WASM: no nested procedures')
    curlev = curlev + 1
    asm.append('(func $' + ident + ' ' +
               ' '.join('(param $' + e.name + ' i32)' for e in fp) + ' ' + 
               ' '.join('(result i32)' for e in rp) +
               ('\n' if len(rp) > 0 else '') +
               '\n'.join('(local $' + e.name + ' i32)' for e in rp))
    return rp

def genProcEntry(ident, para, local):
    pl = (para if para else []) + local
    print("====genProcEntry pl: ", pl)
    if any(type(l) == Var and type(l.tp) in (Array, PartialArray, Record) for l in pl):
        asm.append('(local $_fp i32)')
        asm.append('global.get $_memsize')
        asm.append('local.set $_fp')
        if len(asmPartArr) != 0: asm.extend(asmPartArr)
    for l in pl:
        if type(l) == Var and type(l.tp) in (Array, Record):
            asm.append('global.get $_memsize')
            asm.append('i32.const ' + str(l.tp.size))
            asm.append('i32.add')
            asm.append('local.tee $' + l.name)
            asm.append('global.set $_memsize')
        elif type(l) == Var and type(l.tp) == PartialArray:
            asm.append('global.get $_memsize')
            asm.append('local.tee $' + l.name)
            asm.append('i32.const ' + str(l.tp.size))
            asm.append('i32.add')
            asm.append('global.set $_memsize')
    print("====genProcEntry asm: ", asm)

def genProcExit(x, para, local):
    global curlev
    curlev = curlev - 1
    if any(type(l) == Var and type(l.tp) in (Array, PartialArray, Record) for l in local):
        asm.append('local.get $_fp')
        asm.append('global.set $_memsize')
    if para: asm.append('\n'.join('local.get $' + e.name for e in para))
    asm.append(')')

def genActualPara(ap, fp, n):
    print("======genActualPara ap: ", ap, ", fp: ", fp, ", n: ", n)
    if type(ap) is tuple:
        if type(ap[0]) == Var:
            loadItem(ap[0])
        else:
            loadItem(ap)
    elif ap.tp in {Int, Bool} or type(ap.tp) == Set: loadItem(ap)
    else: # a.tp is Array, Record
        if ap.lev > 0: asm.append('local.get $' + ap.name)
        elif ap.lev == MemAbs: asm.append('i32.const ' + str(ap.adr))
        elif ap.lev != Stack: mark('WASM: actual parameter?')

def genCall(rp, pr, ap): # result (or None), procedure, actual parameters
    asm.append('call $' + pr.name)
    for r in reversed(rp): y = Var(Int); y.lev = Stack; genAssign(r, y)

def genRead(x):
    asm.append('call $read')
    y = Var(Int); y.lev = Stack; genAssign(x, y)

def genWrite(x):
    asm.append('call $write')

def genWriteln():
    asm.append('call $writeln')

def genSeq(x, y):
    pass

def genThen(x):
    loadItem(x)
    asm.append('if')
    return x

def genIfThen(x, y):
    asm.append('end')

def genElse(x, y):
    asm.append('else')

def genIfElse(x, y, z):
    asm.append('end')

def genWhile():
    asm.append('loop')

def genDo(x):
    loadItem(x)
    asm.append('if')
    return x

def genWhileDo(t, x, y):
    asm.append('br 1')
    asm.append('end')
    asm.append('end')